In [1]:
import pymysql
import json
from datetime import datetime

# Sample JSON input
json_input = '''
{
    "name": "Sliced Cake",
    "brand": "Southern Bakeries",
    "product_type": "Cake",
    "size": "105 g",
    "expiry": "09/07/25",
    "price": "40.00",
    "sku": "8906005610942"
}
'''

# MySQL credentials
MYSQL_HOST = "localhost"
MYSQL_USER = "root"           # <-- replace this
MYSQL_PASSWORD = "ritik@01A"   # <-- replace this
DATABASE_NAME = "scanner"                # 👈 database name changed here
TABLE_NAME = "products"

# Function to create database and table if not exist
def setup_mysql():
    conn = pymysql.connect(
        host=MYSQL_HOST,
        user=MYSQL_USER,
        password=MYSQL_PASSWORD
    )
    try:
        with conn.cursor() as cursor:
            cursor.execute(f"CREATE DATABASE IF NOT EXISTS {DATABASE_NAME}")
        conn.commit()
    finally:
        conn.close()

    conn = pymysql.connect(
        host=MYSQL_HOST,
        user=MYSQL_USER,
        password=MYSQL_PASSWORD,
        database=DATABASE_NAME
    )
    try:
        with conn.cursor() as cursor:
            cursor.execute(f"""
                CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    name VARCHAR(255),
                    brand VARCHAR(255),
                    product_type VARCHAR(100),
                    size VARCHAR(50),
                    expiry DATE,
                    price DECIMAL(10,2),
                    sku VARCHAR(50)
                )
            """)
        conn.commit()
    finally:
        conn.close()

# Function to insert product data into MySQL
def insert_product(json_input):
    if isinstance(json_input, str):
        data = json.loads(json_input)
    else:
        data = json_input

    # Convert expiry
    try:
        expiry_date = datetime.strptime(data["expiry"], "%d/%m/%y").strftime("%Y-%m-%d")
    except:
        expiry_date = None

    conn = pymysql.connect(
        host=MYSQL_HOST,
        user=MYSQL_USER,
        password=MYSQL_PASSWORD,
        database=DATABASE_NAME
    )
    try:
        with conn.cursor() as cursor:
            sql = f"""
            INSERT INTO {TABLE_NAME} (name, brand, product_type, size, expiry, price, sku)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            """
            cursor.execute(sql, (
                data.get("name"),
                data.get("brand"),
                data.get("product_type"),
                data.get("size"),
                expiry_date,
                data.get("price"),
                data.get("sku")
            ))
        conn.commit()
        print("✅ Product inserted successfully into scanner.db!")
    finally:
        conn.close()

# Run setup and insert
setup_mysql()
insert_product(json_input)


✅ Product inserted successfully into scanner.db!


## TO GET THE REQUEST 

In [2]:
from fastapi import FastAPI, HTTPException
import pymysql

app = FastAPI()

DB_CONFIG = {
    "host": "localhost",
    "user": "root",
    "password": "ritik@01A",
    "database": "scanner"
}

def get_db_connection():
    return pymysql.connect(**DB_CONFIG)

@app.get("/product-history")
def get_product_history():
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM products ORDER BY id DESC")
        rows = cursor.fetchall()
        cursor.close()
        conn.close()

        keys = ["id", "name", "brand", "product_type", "size", "expiry", "price", "sku"]
        data = [dict(zip(keys, row)) for row in rows]

        return {"status": "success", "data": data}

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Failed to retrieve product history: {str(e)}")


In [ ]:
# ⚠️ Run this in a Jupyter Notebook Cell

from fastapi import FastAPI, HTTPException
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
import pymysql
import uvicorn
from threading import Thread

# Allow nested event loops (needed in notebooks)
nest_asyncio.apply()

# Setup FastAPI app
app = FastAPI()

# Allow CORS if needed
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# DB Config
DB_CONFIG = {
    "host": "localhost",
    "user": "root",
    "password": "ritik@01A",
    "database": "scanner"
}

# Connection function
def get_db_connection():
    return pymysql.connect(**DB_CONFIG)

# GET Endpoint
@app.get("/product-history")
def get_product_history():
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM products ORDER BY id DESC")
        rows = cursor.fetchall()
        cursor.close()
        conn.close()

        keys = ["id", "name", "brand", "product_type", "size", "expiry", "price", "sku"]
        data = [dict(zip(keys, row)) for row in rows]

        return {"status": "success", "data": data}
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Failed to retrieve product history: {str(e)}")

# Run server in a background thread
def run_app():
    uvicorn.run(app, host="127.0.0.1", port=8000)

Thread(target=run_app).start()


INFO:     Started server process [20032]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:53484 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:53485 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:53486 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:53671 - "GET /product-history HTTP/1.1" 200 OK
INFO:     127.0.0.1:56843 - "GET /product-history HTTP/1.1" 200 OK
INFO:     127.0.0.1:56846 - "GET /product-history HTTP/1.1" 200 OK
INFO:     127.0.0.1:56846 - "GET /product-history HTTP/1.1" 200 OK
INFO:     127.0.0.1:56846 - "GET /product-history HTTP/1.1" 200 OK
